# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [29]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])

import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import pickle

from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('messages', con = engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
y = df.drop(columns = ['id', 'message', 'original', 'genre'])

### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
def model_pipeline_rfc():
    pipeline_clf = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 200, random_state = 42)))
    ])
    
    return pipeline_clf

def model_pipeline_abc():
    pipeline_abc = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])
    
    return pipeline_abc

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.33)

In [7]:
model_rfc = model_pipeline_rfc()
model_rfc.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [8]:
model_abc = model_pipeline_abc()
model_abc.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
target_names = list(y_train.columns)

In [10]:
def display_results(y_test, y_pred):
    #Testing the model
    # Printing the classification report for each label
    i = 0
    for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
    accuracy = (y_pred == y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

In [11]:
y_pred_rfc = model_rfc.predict(X_test)
display_results(y_test, y_pred_rfc)

Feature 1: related
             precision    recall  f1-score   support

          0       0.62      0.33      0.43      2054
          1       0.81      0.94      0.87      6534
          2       0.62      0.12      0.21        64

avg / total       0.76      0.79      0.76      8652

Feature 2: request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      7180
          1       0.83      0.37      0.52      1472

avg / total       0.88      0.88      0.86      8652

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      8652

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      5107
          1       0.74      0.51      0.60      3545

avg / total       0.73      0.73      0.71      8652

Feature 5

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
y_pred_abc = model_abc.predict(X_test)
display_results(y_test, y_pred_abc)

Feature 1: related
             precision    recall  f1-score   support

          0       0.57      0.26      0.35      2054
          1       0.80      0.94      0.86      6534
          2       0.25      0.11      0.15        64

avg / total       0.74      0.77      0.73      8652

Feature 2: request
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      7180
          1       0.78      0.55      0.64      1472

avg / total       0.89      0.90      0.89      8652

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.09      0.03      0.04        38

avg / total       0.99      0.99      0.99      8652

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      5107
          1       0.75      0.58      0.66      3545

avg / total       0.75      0.75      0.74      8652

Feature 5

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
model_pipeline_rfc().get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fee4e503400>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [14]:
model_pipeline_abc().get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fee4e503400>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [15]:
parameters_rfc = {'tfidf__use_idf': (True, False),
              'clf__estimator__n_estimators': [50, 100], 
              'clf__estimator__random_state': [42]}

rfc = model_pipeline_rfc()

cv_rfc = GridSearchCV(rfc, param_grid = parameters_rfc)
cv_rfc

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__random_state': [42]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [16]:
parameters_abc = {
    'clf__estimator__learning_rate': [0.1, 0.3],
        'clf__estimator__n_estimators': [100, 200]
    }

abc = model_pipeline_abc()

cv_abc = GridSearchCV(abc, param_grid = parameters_abc, cv = 3, scoring = 'f1_weighted', verbose = 3)
cv_abc

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__random_state': [42], 'clf__estimator__learning_rate': [0.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
cv_rfc.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__random_state': [42]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [18]:
cv_abc.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__random_state': [42], 'clf__estimator__learning_rate': [0.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [19]:
y_pred_cvrfc = cv_rfc.predict(X_test)
display_results(y_test, y_pred_cvrfc)

Feature 1: related
             precision    recall  f1-score   support

          0       0.71      0.25      0.37      2054
          1       0.80      0.97      0.87      6534
          2       0.57      0.06      0.11        64

avg / total       0.77      0.79      0.75      8652

Feature 2: request
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      7180
          1       0.90      0.43      0.58      1472

avg / total       0.90      0.89      0.88      8652

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      8652

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.76      0.88      0.82      5107
          1       0.78      0.60      0.68      3545

avg / total       0.77      0.77      0.76      8652

Feature 5

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
y_pred_cvabc = cv_abc.predict(X_test)
display_results(y_test, y_pred_cvabc)

Feature 1: related
             precision    recall  f1-score   support

          0       0.63      0.20      0.30      2054
          1       0.79      0.96      0.87      6534
          2       0.32      0.09      0.14        64

avg / total       0.75      0.77      0.73      8652

Feature 2: request
             precision    recall  f1-score   support

          0       0.91      0.98      0.94      7180
          1       0.81      0.53      0.64      1472

avg / total       0.89      0.90      0.89      8652

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.25      0.03      0.05        38

avg / total       0.99      1.00      0.99      8652

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.76      0.87      0.81      5107
          1       0.77      0.60      0.67      3545

avg / total       0.76      0.76      0.75      8652

Feature 5

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
model_pipeline_abc2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(
        AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced'))
    ))
])

parameters_abc2 = {
    'clf__estimator__learning_rate': [0.1, 0.3],
    'clf__estimator__n_estimators': [100, 200]
}

cv_ada = GridSearchCV(estimator=model_pipeline_abc2, param_grid=parameters_abc2)

In [22]:
cv_ada.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...er='best'),
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__learning_rate': [0.1, 0.3], 'clf__estimator__n_estimators': [100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [23]:
cv_ada.best_params_

{'clf__estimator__learning_rate': 0.3, 'clf__estimator__n_estimators': 200}

In [24]:
y_pred_ada = cv_ada.predict(X_test)
display_results(y_test, y_pred_ada)

Feature 1: related
             precision    recall  f1-score   support

          0       0.41      0.62      0.49      2054
          1       0.86      0.70      0.77      6534
          2       0.20      0.64      0.30        64

avg / total       0.75      0.68      0.70      8652

Feature 2: request
             precision    recall  f1-score   support

          0       0.95      0.88      0.91      7180
          1       0.57      0.77      0.65      1472

avg / total       0.88      0.86      0.87      8652

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      0.97      0.98      8614
          1       0.03      0.21      0.05        38

avg / total       0.99      0.97      0.98      8652

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.79      0.82      0.80      5107
          1       0.72      0.69      0.71      3545

avg / total       0.76      0.76      0.76      8652

Feature 5

In [31]:
pipeline2 = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('best', TruncatedSVD()),
            ('tfidf', TfidfTransformer())]))])), 
    ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42)))
])

In [32]:
pipeline2.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip...ol=0.0)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))]))],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=42),
              n_jobs=1))], 'features': FeatureUnion(n_jobs=1,
        transformer_list=[('text_pipeline', Pipeline(memory=None,
      steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         d

In [34]:
parameters2 = {'clf__estimator__n_estimators': [100, 200, 300], 
              'clf__estimator__random_state': [42],
             'clf__estimator__learning_rate': [0.1]} 

cv2 = GridSearchCV(pipeline2, param_grid = parameters2, cv = 10,
                  refit = True, verbose = 1, return_train_score = True, n_jobs = -1)
cv2

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...timator=None,
          learning_rate=1.0, n_estimators=50, random_state=42),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [100, 200, 300], 'clf__estimator__random_state': [42], 'clf__estimator__learning_rate': [0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [35]:
best_ada = cv2.fit(X_train, y_train)
best_ada.best_params_

Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 52.6min finished


{'clf__estimator__learning_rate': 0.1,
 'clf__estimator__n_estimators': 100,
 'clf__estimator__random_state': 42}

In [36]:
y_pred_bada = best_ada.predict(X_test)
display_results(y_test, y_pred_bada)

Feature 1: related
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      2054
          1       0.76      1.00      0.86      6534
          2       0.00      0.00      0.00        64

avg / total       0.57      0.76      0.65      8652

Feature 2: request
             precision    recall  f1-score   support

          0       0.83      1.00      0.91      7180
          1       0.00      0.00      0.00      1472

avg / total       0.69      0.83      0.75      8652

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      8652

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.60      0.93      0.73      5107
          1       0.53      0.11      0.18      3545

avg / total       0.57      0.60      0.51      8652

Feature 5

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [ ]:
with open('cv_abc.pkl', 'wb') as file:
    pickle.dump(cv_ada, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.